In [1]:
import sys
import numpy
import dateutil.parser as parser
import csv
import pickle
import scipy.stats
import os
import re
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, WeekdayLocator, DateFormatter, drange
from dateutil.relativedelta import relativedelta
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
%matplotlib inline
from sklearn.linear_model import BayesianRidge

In [2]:
nonfarm_org = [161.0, 44.0, 332.0, 249.0, 307.0, 74.0, 32.0, 132.0, 162.0, 346.0, 65.0, 129.0, 134.0, 239.0, 134.0, 363.0, 175.0, 245.0, 373.0, 196.0, 67.0, 84.0, 337.0, 159.0, 277.0, 315.0, 280.0, 182.0, 23.0, 77.0, 207.0, 184.0, 157.0, 2.0, 210.0, 171.0, 238.0, 88.0, 188.0, 78.0, 144.0, 71.0, -33.0, -16.0, 85.0, 82.0, 118.0, 97.0, 15.0, -86.0, -80.0, -214.0, -182.0, -172.0, -210.0, -259.0, -452.0, -474.0, -765.0, -697.0, -798.0, -701.0, -826.0, -684.0, -354.0, -467.0, -327.0, -216.0, -227.0, -198.0, -6.0, -283.0, 18.0, -50.0, 156.0, 251.0, 516.0, -122.0, -61.0, -42.0, -57.0, 241.0, 137.0, 71.0, 70.0, 168.0, 212.0, 322.0, 102.0, 217.0, 106.0, 122.0, 221.0, 183.0, 164.0, 196.0, 360.0, 226.0, 243.0, 96.0, 110.0, 88.0, 160.0, 150.0, 161.0, 225.0, 203.0, 214.0, 197.0, 280.0, 141.0, 203.0, 199.0, 201.0, 149.0, 202.0, 164.0, 237.0, 274.0, 84.0, 144.0, 222.0, 203.0, 304.0, 229.0, 267.0, 243.0, 203.0,271,261,423,329,201,266,119,221,280]
nonfarm_payroll = nonfarm_org[-40:]

In [3]:
import pickle
monthly_data = {}
feature_names = []
with open('monthly_features.pickle', 'rb') as f:
    monthly_features = pickle.load(f)
    for k,v in monthly_features.items():
        feature_names = list(v.keys())
        monthly_data[k] = list(v.values())


In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame(columns=feature_names)
df = pd.DataFrame.from_dict(monthly_data, orient="index")
df.columns = feature_names
df = df.sort()
df_norm = (df - df.mean()) / (df.max() - df.min())

In [6]:
df_norm

,total,Retail,Banking,Healthcare,Chemicals,Media,age>50,Education,age26-35,Communications,...,Hospitality,Manufacturing,Construction,gender-ratio,Staffing,Electronics,age<25,Assisted Living,Legal,Engineering
2012-01,-0.399217,0.180288,0.147538,0.415915,0.066673,-0.375185,0.420692,0.694170,-0.093861,0.706194,...,0.481528,0.537614,0.369590,-0.047757,0.619489,-0.459235,-0.460632,0.571900,-0.131116,-0.088850
2012-02,-0.384988,0.233324,0.318304,0.290606,0.240386,-0.422725,0.423589,0.429805,0.062927,0.408298,...,0.616129,0.351193,0.462120,0.144401,0.625701,0.234481,-0.552302,0.451250,0.418033,-0.019470
2012-03,-0.369956,0.175033,0.309128,0.292468,0.168661,-0.402514,0.209665,0.413689,-0.073920,0.299351,...,0.609334,0.444811,0.590851,0.257828,0.647366,0.085604,-0.233260,0.723397,0.695904,0.097928
2012-04,-0.345577,-0.143134,0.121612,0.179222,0.275330,-0.443102,-0.172508,0.222792,-0.200393,0.146616,...,0.499335,0.383144,0.637677,0.332513,0.277734,0.299661,0.262133,0.110919,0.021575,0.310772
2012-05,-0.310282,-0.018021,-0.359290,0.772733,0.044585,-0.334834,0.343682,0.167071,-0.695952,0.064593,...,0.154821,0.224765,0.161910,0.073712,0.133204,-0.268098,0.342463,0.438737,0.146058,0.218746
2012-06,-0.319977,0.178553,0.256860,0.261764,0.024455,-0.156287,-0.301070,0.345899,-0.307293,0.086660,...,0.123014,0.315734,0.054266,0.049025,0.248719,0.092470,0.447698,0.205893,0.118033,0.045758
2012-07,-0.314688,-0.036812,0.165173,0.208670,0.304390,-0.164952,-0.232131,0.280950,0.005141,0.040585,...,-0.007794,0.058024,0.397995,-0.173186,0.299396,0.518099,0.160403,0.101355,0.120069,0.109118
2012-08,-0.300167,-0.041902,-0.074801,0.175752,0.770396,0.075582,-0.165041,0.260445,0.003098,0.181160,...,0.030517,0.139821,0.160473,-0.294507,0.316077,0.393578,0.090767,0.346839,-0.047171,0.045386
2012-09,-0.300556,-0.156812,-0.240532,0.072268,0.310721,0.205752,0.038523,0.250332,-0.087037,0.340480,...,-0.032490,-0.025428,0.128847,-0.205734,0.325080,-0.000298,-0.011862,0.239001,-0.177778,0.187026
2012-10,-0.275503,0.317219,-0.111123,0.063545,0.229138,-0.007101,0.394512,0.133150,-0.210683,0.313845,...,-0.113164,0.096373,0.312131,-0.222459,0.327061,0.061422,-0.140224,0.142691,-0.225833,0.093945


In [7]:
df_norm['nonfarm'] = nonfarm_payroll

In [8]:
from sklearn import linear_model

In [9]:
import numpy as np

In [74]:
def shift_two_series(s0, s1, shift):
    if shift > 0:
        return s0[:-shift], s1[shift:]
    elif shift < 0:
        return s0[-shift:], s1[:shift]
    else:
        return s0, s1

def fit_and_test(train, test, shift):
    # predictor = linear_model.Lasso(alpha=0.1,max_iter=10000)
    predictor = BayesianRidge()
    x, y = shift_two_series(train[feature_names].values,train['nonfarm'].values, shift)
    predictor.fit(x,y)
    if shift > 0:
        prediction = predictor.predict(test[feature_names].values[:-shift])
        mean_error = abs(np.mean((test['nonfarm'].values[shift:] - prediction)/test['nonfarm'].values[shift:]))
    elif shift < 0:
        prediction = predictor.predict(test[feature_names].values[-shift:])
        mean_error = abs(np.mean((test['nonfarm'].values[:shift] - prediction)/test['nonfarm'].values[:shift]))
    else:
        prediction = predictor.predict(test[feature_names].values)
        mean_error = abs(np.mean((test['nonfarm'].values - prediction)/test['nonfarm'].values))
        
    #print(test['nonfarm'].values)
    #print(prediction)
    #print((test['nonfarm'].values-prediction)/test['nonfarm'].values)
    #print(mean_error)
    return mean_error

In [75]:
train = df_norm[:30]
test = df_norm[-10:]
fit_and_test(train, test, 2)

0.010560602775896211

In [93]:
def scan1():
    for n in range(25, 40):
        train = df_norm[n-24:n]
        test = df_norm[n:n+1]
        yield fit_and_test(train, test, 0)

In [94]:
r = list(scan1())
np.mean(r)

0.20800330462318842

In [132]:
def linearBayesianEstimate(history_x, history_y, x):
    T = 4
    l2norm_squre = np.sum(np.square(x - history_x),axis=1)
    y = sum(history_y * l2norm_squre) / sum(l2norm_squre)
    return y

def test_LBE(train, test):
    x, y = train[feature_names].values,train['nonfarm'].values
    prediction = linearBayesianEstimate(x, y, test[feature_names].values)
    error = abs((test['nonfarm'].values - prediction)/test['nonfarm'].values)
    return error

def scan_LBE():
    for n in range(25, 40):
        train = df_norm[:n]
        test = df_norm[n:n+1]
        yield test_LBE(train, test)

In [133]:
r = list(scan_LBE())
np.mean(r)

0.29292520698669311